In [ ]:
from get_features import *
from tqdm import notebook

In [ ]:
one = ONE()

In [ ]:
lab_number_map = get_lab_number_map()

In [ ]:
has_GPIO, no_GPIO = get_valid_eids()

In [ ]:
traj = get_traj(has_GPIO, no_GPIO)

In [ ]:
ts_directory = os.path.join('.', "new_timestamps")
if not os.path.exists(ts_directory):
    os.makedirs(ts_directory)
for i in notebook.tqdm(range(len(traj))):
    eid = traj[i]['session']['id']
    try:
        get_new_timestamps(eid, 'left', ts_directory)
        get_new_timestamps(eid, 'right', ts_directory)
    except:
        print('error')

In [ ]:
acronym_dict = get_acronym_dict(one, traj, has_GPIO, no_GPIO, ts_directory)

In [ ]:
# acronym_dict = np.load('../acronym_dict_new.npy', allow_pickle=True)

In [ ]:
feature_list = []
output_list = []
trial_length_array_list = []
cluster_number_list = []
session_list = []
nan_trials = []
for i in notebook.tqdm(range(len(traj))):
    feature, output, trial_length_array, cluster_numbers, nan_idx, success = featurize(i, traj[i], one, lab_number_map, 
                                                                                       acronym_dict.tolist(), has_GPIO, no_GPIO, ts_directory)
    if not success:
        print('not successful')
        continue
    else:
        print('successful')
    feature_list.append(feature)
    output_list.append(output)
    trial_length_array_list.append(trial_length_array)
    cluster_number_list.append(cluster_numbers)
    session_list.append(traj[i])
    nan_trials.append(nan_idx)

In [ ]:
for i in range(len(feature_list)):
    print(feature_list[i].shape)

In [ ]:
for i in range(len(output_list)):
    print(output_list[i].shape)

In [ ]:
topk_list = []
for i in range(len(output_list)):
    mean_fr = output_list[i].mean(2).mean(0)
    top_k = mean_fr.argsort()[::-1][:250]
    topk_list.append(top_k)

In [ ]:
truncated_feature_list = []
for i in range(len(feature_list)):
    top_k = topk_list[i]
    feature = feature_list[i][:,top_k]
    for k in range(10):
        feature[k,:,:,0] = 10*i + k
    truncated_feature_list.append(feature)

In [ ]:
truncated_feature = np.asarray(truncated_feature_list)
print(truncated_feature.shape)

In [ ]:
np.save('truncated_feature_original.npy', truncated_feature)

In [ ]:
seqlen_list = []
for i in range(len(trial_length_array_list)):
    top_k = topk_list[i]
    seqlen_list.append(trial_length_array_list[i][top_k])

In [ ]:
seqlen_array = np.asarray(seqlen_list)
print(seqlen_array.shape)

In [ ]:
np.save('seqlen.npy', seqlen_array)

In [ ]:
# normalize truncated_feature: dlc features + xyz location
# first normalize xyz
x_max = truncated_feature[:,:,:,:,xyz_offset].max()
x_min = truncated_feature[:,:,:,:,xyz_offset].min()
y_max = truncated_feature[:,:,:,:,xyz_offset+1].max()
y_min = truncated_feature[:,:,:,:,xyz_offset+1].min()
z_max = truncated_feature[:,:,:,:,xyz_offset+2].max()
z_min = truncated_feature[:,:,:,:,xyz_offset+2].min()

truncated_feature[:,:,:,:,xyz_offset] = 0.1 + 0.9*(truncated_feature[:,:,:,:,xyz_offset] - x_min) / (x_max - x_min)
truncated_feature[:,:,:,:,xyz_offset+1] = 0.1 + 0.9*(truncated_feature[:,:,:,:,xyz_offset+1] - y_min) / (y_max - y_min)
truncated_feature[:,:,:,:,xyz_offset+2] = 0.1 + 0.9*(truncated_feature[:,:,:,:,xyz_offset+2] - z_min) / (z_max - z_min)

# next normalize dlc features
for i in range(stimOnOff_offset - left_dlc_offset):
    idx = left_dlc_offset+i
    
    feature_min = truncated_feature[:,:,:,:,idx].min()
    feature_max = truncated_feature[:,:,:,:,idx].max()
    
    truncated_feature[:,:,:,:,idx] = -1 + 2*(truncated_feature[:,:,:,:,idx] - feature_min) / (feature_max - feature_min)
    
# next normalize wheel
wheel_min = truncated_feature[:,:,:,:,wheel_offset].min()
wheel_max = truncated_feature[:,:,:,:,wheel_offset].max()

truncated_feature[:,:,:,:,wheel_offset] = -1 + 2*(truncated_feature[:,:,:,:,wheel_offset] - wheel_min) / (wheel_max - wheel_min)


# next normalize lick
lick_min = truncated_feature[:,:,:,:,lick_offset].min()
lick_max = truncated_feature[:,:,:,:,lick_offset].max()

truncated_feature[:,:,:,:,lick_offset] = (truncated_feature[:,:,:,:,lick_offset] - lick_min) / (lick_max - lick_min)

# next normalize max_ptp
max_ptp_min = truncated_feature[:,:,:,:,max_ptp_offset].min()
max_ptp_max = truncated_feature[:,:,:,:,max_ptp_offset].max()

truncated_feature[:,:,:,:,max_ptp_offset] = 0.1 + 0.9*(truncated_feature[:,:,:,:,max_ptp_offset] - max_ptp_min) / (max_ptp_max - max_ptp_min)

# next normalize wf_width
wf_width_min = truncated_feature[:,:,:,:,wf_width_offset].min()
wf_width_max = truncated_feature[:,:,:,:,wf_width_offset].max()

truncated_feature[:,:,:,:,wf_width_offset] = 0.1 + 0.9*(truncated_feature[:,:,:,:,wf_width_offset] - wf_width_min) / (wf_width_max - wf_width_min)

In [ ]:
np.save('truncated_feature_normalized.npy', truncated_feature)

In [ ]:
truncated_target_list = []
for i in range(len(output_list)):
    top_k = topk_list[i]
    output = output_list[i][:,top_k]
    truncated_target_list.append(output)

In [ ]:
truncated_target = np.asarray(truncated_target_list)
print(truncated_target.shape)

In [ ]:
np.save('truncated_output.npy', truncated_target)